<a href="https://colab.research.google.com/github/ZhenghaoXiao32/Frequent-Pattern-Mining-for-PLCB/blob/master/Copy_of_fpm_plcb_prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module and Dependencies

In [0]:
# Install mlxtend
!pip install mlxtend
!pip install mlxtend --upgrade --no-deps

     |████████████████████████████████| 1.3MB 4.8MB/s 
  Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [0]:
# Point Colaboratory to your Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Prototypes with Pandas and mlxtend

## Data Preparation

In [0]:
product_csv = '/content/gdrive/My Drive/PLCB/product.csv'
sales_csv = '/content/gdrive/My Drive/PLCB/sales.csv'

In [0]:

import pandas as pd

### Read the dataset

Read the transaction dataset with columns we need:

In [0]:
pdDF_txn = pd.read_csv(sales_csv, usecols=['Transaction_Id', 'Item_Key'], 
                       dtype={'Transaction_Id': 'str', 'Item_Key': 'str'})

Have a look of the transaction data:

In [0]:
pdDF_txn.head()

,Transaction_Id,Item_Key
0,25627392735,206905
1,25623860995,426808
2,25662837538,437850
3,25646360028,484964
4,25598222519,168641


In [0]:
pdDF_txn.rename(columns={"Item_Key": "ITEM_KEY"}, inplace=True)

In [0]:
pdDF_txn.head()

,Transaction_Id,ITEM_KEY
0,25627392735,206905
1,25623860995,426808
2,25662837538,437850
3,25646360028,484964
4,25598222519,168641


In [0]:
pdDF_txn.tail()

,Transaction_Id,ITEM_KEY
999995,25579969932,622
999996,25623734448,320266
999997,25652976000,317345
999998,25650038713,218995
999999,25636212855,319926


In [0]:
txn_sizes = pdDF_txn.Transaction_Id.value_counts()

In [0]:
sum(txn_sizes > 1)

13409

In [0]:
print(" The percentage of transactions that has more than 1 item is {}%.".format(13409/1000000 * 100))

 The percentage of transactions that has more than 1 item is 1.3409%.


Have a look of the products dataset:

In [0]:
sample_products = pd.read_csv(product_csv, nrows=2)
sample_products

,ITEM_KEY,ITEM_IDNT,LEVEL1_KEY,LEVEL2_KEY,LEVEL3_KEY,ITEM_DESC,STND_UOM_CDE,PACKAGE_UOM,PACKAGE_SIZE,STND_UOM_DESC,INV_IND,SBCLASS_IDNT,SBCLASS_DESC,CLASS_IDNT,CLASS_DESC,DEPT_IDNT,DEPT_DESC,GRP_IDNT,GRP_DESC,DIV_IDNT,DIV_DESC,CMPY_IDNT,CMPY_DESC,CMPY_START_DT,CMPY_END_DT,LIQUID_VOLUME,LIQUID_VOLUME_UOM,CASE_QTY,RLC_TYPE_UDA_HEAD_KEY,RLC_TYPE_UDA_HEAD_IDNT,RLC_TYPE_UDA_HEAD_DESC,ITEM_QUAL_UDA_HEAD_KEY,ITEM_QUAL_UDA_HEAD_IDNT,ITEM_QUAL_UDA_HEAD_DESC,RLC_TYPE_UDA_DTL_KEY,RLC_TYPE_UDA_DTL_IDNT,RLC_TYPE_UDA_DTL_DESC,ITEM_QUAL_UDA_DTL_KEY,ITEM_QUAL_UDA_DTL_IDNT,ITEM_QUAL_UDA_DTL_DESC,MKT_CAT_UDA_HEAD_KEY,MKT_CAT_UDA_HEAD_IDNT,MKT_CAT_UDA_HEAD_DESC,PRC_SEG_UDA_HEAD_KEY,PRC_SEG_UDA_HEAD_IDNT,PRC_SEG_UDA_HEAD_DESC,MKT_CAT_UDA_DTL_KEY,MKT_CAT_UDA_DTL_IDNT,MKT_CAT_UDA_DTL_DESC,PRC_SEG_UDA_DTL_KEY,PRC_SEG_UDA_DTL_IDNT,PRC_SEG_UDA_DTL_DESC,SUPP_IDNT,VELOCITY_CATEGORY,Chairmans,Alt_Liq_Vol_UOM,Alt_Liq_Vol
0,511471,79069,511471,-1,-1,Double Canyon Cabernet Sauvignon Horse Heaven ...,EA,NaN,0,Each,Y,4120,WA or OR Cabernet Sauvignon,4100,Red Table Wine,400,Luxury Wine,2,Luxury,1,Stock,1,PLCB,2003-07-05 00:00:00,0000-00-00 00:00:00,25.4,FZ,12,259,56.0,Item Type,260,57.0,Item Qualifier,32562,733.0,CABERNET SAUVIGNON (WA),32682,2.0,S- Luxury,269,65.0,Marketing Category,262,59.0,Price Segments,33211,511.0,US CABERNET,32699,3.0,Premium,Not Specified,Not Specified,2,2019,0
1,502792,3760280780012,502617,502792,-1,Chateau Gruaud Larose Saint Julien 2016,EA,NaN,0,Each,Y,4154,France Bordeaux,4100,Red Table Wine,400,Luxury Wine,2,Luxury,1,Stock,1,PLCB,2003-07-05 00:00:00,0000-00-00 00:00:00,0.0,NaN,0,0,NaN,Not Specified,0,NaN,Not Specified,0,NaN,Not Specified,0,NaN,Not Specified,0,NaN,Not Specified,0,NaN,Not Specified,0,NaN,Not Specified,0,NaN,Not Specified,Not Specified,Not Specified,2,2019,0


#### Read product dataset use Pandas

Read the product dataset with columns we need:

In [0]:
pdDF_products = pd.read_csv(product_csv, usecols=['ITEM_KEY', 'ITEM_DESC', 'SBCLASS_DESC', 'CLASS_DESC'], 
                            dtype={'ITEM_KEY': 'str', 'ITEM_DESC': 'str', 'SBCLASS_DESC': 'str', 'CLASS_DESC': 'str'})

Check if the items are unique:

In [0]:
pdDF_products.ITEM_KEY.duplicated().sum()

0

## Data cleaning

### Concatenate subclasses and classes in product dataset

Have a look of subclasses:

In [0]:
pdDF_products.SBCLASS_DESC.value_counts()


Imported                  125022
Domestic                   63055
Italy North                19979
France Burgundy            18710
Spain                      16682
                           ...  
Register Tape                  4
Salesman Sample Spirit         2
Salesman Sample                2
Rose                           2
No Subclass                    1
Name: SBCLASS_DESC, Length: 158, dtype: int64

In [0]:
pdDF_products.SBCLASS_DESC.value_counts().head(20)

Imported                         125022
Domestic                          63055
Italy North                       19979
France Burgundy                   18710
Spain                             16682
Italy Central                     16231
California Cabernet Sauvignon     15504
Australia                         12009
California Chardonnay             11891
France Rhone                       8945
California Pinot Noir              8867
Argentina                          8824
France Bordeaux                    8722
California Red Blend               7479
Italy South                        6722
Chile                              6590
Unflavored                         6204
France South Southwest             6101
France Loire                       6007
Flavored                           5785
Name: SBCLASS_DESC, dtype: int64

In [0]:
pdDF_products.CLASS_DESC.value_counts()

Red Table Wine           343293
White Table Wine          85094
Sparkling                 17835
Dessert                   12480
Whiskey                   11406
Miscellaneous             10166
Rose Table Wine            8961
Liquor or Cordials         7004
Vodka                      6517
Tequila                    4424
Rum                        3860
Brandy Cognac              3764
Box                        1888
Gin                        1452
Rose or Blush              1146
Cocktails  Mixers          1132
Ready to Drink              884
Supplies                    168
Gift Cards                   94
Bags                         17
Towels                        5
Register Tape                 4
Saleman Sample Spirit         2
Salesman Sample               2
No Class                      1
Name: CLASS_DESC, dtype: int64

To make the product descriptions more meaningful, we decided to concatenate subclasses descriptions and classes descriptions:

In [0]:
pdDF_products['ITEM'] = pdDF_products['SBCLASS_DESC'] + ' ' + pdDF_products['CLASS_DESC']

In [0]:
pdDF_products.head()

,ITEM_KEY,ITEM_DESC,SBCLASS_DESC,CLASS_DESC,ITEM
0,511471,Double Canyon Cabernet Sauvignon Horse Heaven ...,WA or OR Cabernet Sauvignon,Red Table Wine,WA or OR Cabernet Sauvignon Red Table Wine
1,502792,Chateau Gruaud Larose Saint Julien 2016,France Bordeaux,Red Table Wine,France Bordeaux Red Table Wine
2,33274,Richards Wild Irish Rose,Hi Proof Flavored,Miscellaneous,Hi Proof Flavored Miscellaneous
3,295019,ROSENBLUM PASO ROBLES ZINFANDEL ...,California Zinfandel,Red Table Wine,California Zinfandel Red Table Wine
4,505118,Chateau Boutillot Bordeaux Rouge 2015,France Bordeaux,Red Table Wine,France Bordeaux Red Table Wine


In [0]:
df_item = pdDF_products[['ITEM_KEY', 'ITEM', 'CLASS_DESC']]

In [0]:
df_item.head()

,ITEM_KEY,ITEM,CLASS_DESC
0,511471,WA or OR Cabernet Sauvignon Red Table Wine,Red Table Wine
1,502792,France Bordeaux Red Table Wine,Red Table Wine
2,33274,Hi Proof Flavored Miscellaneous,Miscellaneous
3,295019,California Zinfandel Red Table Wine,Red Table Wine
4,505118,France Bordeaux Red Table Wine,Red Table Wine


### Join sales data with cleaned products data

Concatenate new item descriptions with the transactions data:

In [0]:
df_txn = pdDF_txn.join(df_item.set_index('ITEM_KEY'), on='ITEM_KEY')

In [0]:
df_txn.head(20)

,Transaction_Id,ITEM_KEY,ITEM,CLASS_DESC
0,25627392735,206905,American Whiskey,Whiskey
1,25623860995,426808,Flavored Whiskey,Whiskey
2,25662837538,437850,Flavored Brandy Cognac,Brandy Cognac
3,25646360028,484964,Italy Other Sparkling Sparkling,Sparkling
4,25598222519,168641,Canadian Whiskey,Whiskey
5,25621514290,437143,WA or OR Sparkling,Sparkling
6,25605015027,933,Brandy Brandy Cognac,Brandy Cognac
7,25659592060,431231,Other Imports Red Table Wine,Red Table Wine
8,25655803064,503394,Fruit Liquor or Cordials,Liquor or Cordials
9,25642727485,2918,Irish Whiskey,Whiskey


In [0]:
df_txn.ITEM.value_counts()

Unflavored Vodka                                132419
American Whiskey                                 64497
Flavored Vodka                                   43695
California Cabernet Sauvignon Red Table Wine     29953
Canadian Whiskey                                 29207
                                                 ...  
US Other Cabernet Sauvignon Red Table Wine           2
Imported Dessert                                     2
US Other Shiraz or Syrah Red Table Wine              1
US Other Red Blend Red Table Wine                    1
New York Sparkling                                   1
Name: ITEM, Length: 197, dtype: int64

In [0]:
df_txn.ITEM.value_counts().head(50)

Unflavored Vodka                                    132419
American Whiskey                                     64497
Flavored Vodka                                       43695
California Cabernet Sauvignon Red Table Wine         29953
Canadian Whiskey                                     29207
California Chardonnay White Table Wine               28601
Flavored Whiskey                                     27051
Unflavored Rum                                       22218
Beverage Miscellaneous                               21530
California Red Blend Red Table Wine                  21244
Flavored Rum                                         20125
Red Box                                              20088
Schnapps Liquor or Cordials                          19362
Brandy Brandy Cognac                                 19267
Unflavored Gin                                       18394
Spiced Rum                                           17906
California Rose or Blush                             169

In [0]:
df_txn.tail(20)

,Transaction_Id,ITEM_KEY,ITEM,CLASS_DESC
999980,25632604981,483706,Flavored Vodka,Vodka
999981,25636478081,220375,Spiced Rum,Rum
999982,25606078202,477654,France Rose or Blush,Rose or Blush
999983,25660860267,2256,Unflavored Vodka,Vodka
999984,25664726851,503433,Spiced Rum,Rum
999985,25623536925,319720,Cream Liquor or Cordials,Liquor or Cordials
999986,25665824462,390018,Flavored Whiskey,Whiskey
999987,25669415947,322828,France Burgundy White Table Wine,White Table Wine
999988,25669871039,503395,Nut Liquor or Cordials,Liquor or Cordials
999989,25617267789,853,Unflavored Gin,Gin


In [0]:
# save the df_txn 
df_txn.to_csv('df_txn.csv', index=False)
!cp df_txn.csv '/content/gdrive/My Drive/PLCB/'

In [0]:
# Create a dataframe using item_key as items
df_by_itemkey = \
df_txn[['Transaction_Id', 'ITEM_KEY']].groupby('Transaction_Id'\
,as_index=False).agg(lambda x: x.tolist())

In [0]:
df_by_itemkey.head()

,Transaction_Id,ITEM_KEY
0,100000000000000128121,[319409]
1,100000000000000128122,"[318665, 316925]"
2,100000000000000128133,[321130]
3,100000000000000128134,"[142006, 221835]"
4,100000000000000128137,[450950]


In [0]:
# Create a dataframe using item as items
df_by_item = df_txn[['Transaction_Id', 'ITEM']]\
.groupby('Transaction_Id', as_index=False).agg(lambda x: x.tolist())

In [0]:
df_by_item.head()

,Transaction_Id,ITEM
0,100000000000000128121,[White Box]
1,100000000000000128122,"[California Red Blend Red Table Wine, Californ..."
2,100000000000000128133,[New York Red Table Wine]
3,100000000000000128134,"[Beverage Miscellaneous, California Cabernet S..."
4,100000000000000128137,[California Rose or Blush]


In [0]:
# Create a dataframe using class_desc as items
df_by_class = df_txn[['Transaction_Id', 'CLASS_DESC']]\
.groupby('Transaction_Id', as_index=False).agg(lambda x: x.tolist())

In [0]:
df_by_class.head()

,Transaction_Id,CLASS_DESC
0,100000000000000128121,[Box]
1,100000000000000128122,"[Red Table Wine, Red Table Wine]"
2,100000000000000128133,[Red Table Wine]
3,100000000000000128134,"[Miscellaneous, Red Table Wine]"
4,100000000000000128137,[Rose or Blush]


## Frequent Pattern Mining With mlxtend

try with mlxtend:

In [0]:
from mlxtend.preprocessing import TransactionEncoder

### Use ITEM_KEY as items

In [0]:
# using item_key as items
dataset = df_by_itemkey.ITEM_KEY
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df_te = pd.DataFrame(te_ary, columns=te.columns_)
df_te

,100280,1004,1006,1007,100732,1008,1009,100962,100963,100991,1010,101151,1012,1015,101537,101580,101581,101811,101891,101964,101968,1022,1023,1024,1025,1026,102862,102865,103162,103196,1033,1034,103455,1036,104432,104474,1046,1055,105976,1060,...,880,881,883,884,885,889,890,891,900,902,910,917,918,921,924,933,936,946,95080,953,957,962,964,967,97097,97463,97557,98112,982,985,98869,98870,98872,98882,99077,99255,997,99771,99772,999
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985645,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
985646,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,F

In [0]:
from mlxtend.frequent_patterns import fpgrowth, association_rules

In [0]:
frequent_itemsets = fpgrowth(df_te, min_support=0.00001, use_colnames=True)

In [0]:
frequent_itemsets

,support,itemsets
0,0.000158,(319409)
1,0.001912,(318665)
2,0.000234,(316925)
3,0.000217,(321130)
4,0.000578,(221835)
...,...,...
5067,0.000016,(511784)
5068,0.000014,(513157)
5069,0.000022,(513158)
5070,0.000018,(513159)


In [0]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.001)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(842),(3016),0.005023,0.003711,0.000011,0.002222,0.598658,-0.000007,0.998507
1,(3016),(842),0.003711,0.005023,0.000011,0.003007,0.598658,-0.000007,0.997978


In [0]:
# check what are item_key 842 and 3016
df_txn.loc[df_txn['ITEM_KEY'] == "842"]

,Transaction_Id,ITEM_KEY,ITEM,CLASS_DESC
170,25652086727,842,American Whiskey,Whiskey
265,25593936864,842,American Whiskey,Whiskey
302,25642994251,842,American Whiskey,Whiskey
659,25603451855,842,American Whiskey,Whiskey
696,25652859434,842,American Whiskey,Whiskey
...,...,...,...,...
999291,25590848890,842,American Whiskey,Whiskey
999340,25642681837,842,American Whiskey,Whiskey
999532,25671571392,842,American Whiskey,Whiskey
999891,25659018195,842,American Whiskey,Whiskey


In [0]:
df_txn.loc[df_txn['ITEM_KEY'] == "3016"]

,Transaction_Id,ITEM_KEY,ITEM,CLASS_DESC
664,25645588709,3016,Irish Whiskey,Whiskey
842,25661799944,3016,Irish Whiskey,Whiskey
871,25588754366,3016,Irish Whiskey,Whiskey
1160,25651624492,3016,Irish Whiskey,Whiskey
2349,25590506573,3016,Irish Whiskey,Whiskey
...,...,...,...,...
998403,25613988537,3016,Irish Whiskey,Whiskey
998601,25611932782,3016,Irish Whiskey,Whiskey
998994,25587760651,3016,Irish Whiskey,Whiskey
999438,25658989808,3016,Irish Whiskey,Whiskey


In [0]:
rules.confidence.max()

0.0030071077091306723

### USE ITEM as items

In [0]:
dataset = df_by_item.ITEM
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df_te = pd.DataFrame(te_ary, columns=te.columns_)
df_te

,Alcohol Cocktails Mixers,All 4 Packs Red Table Wine,All 4 Packs Rose or Blush,All 4 Packs Sparkling,All 4 Packs White Table Wine,American Whiskey,Anejo or Super Anejo Tequila,Argentina Red Table Wine,Argentina Rose Table Wine,Argentina Rose or Blush,Argentina White Table Wine,Armagnac Brandy Cognac,Asian Miscellaneous,Australia Red Table Wine,Australia Rose Table Wine,Australia Rose or Blush,Australia White Table Wine,Austria Red Table Wine,Austria White Table Wine,Bags Miscellaneous,Barware Miscellaneous,Beverage Miscellaneous,Brandy Brandy Cognac,California Cabernet Sauvignon Red Table Wine,California Chardonnay White Table Wine,California Merlot Red Table Wine,California Other Red Varietals Red Table Wine,California Other White White Table Wine,California Pinot Grigio or Gris White Table Wine,California Pinot Noir Red Table Wine,California Red Blend Red Table Wine,California Rose Table Wine,California Rose or Blush,California Sauvignon Blanc White Table Wine,California Shiraz or Syrah Red Table Wine,California Sparkling,California White Blend White Table Wine,California Zinfandel Red Table Wine,Canadian Whiskey,Carded Items Miscellaneous,...,South Africa Red Table Wine,South Africa White Table Wine,South America Red Table Wine,Spain Red Table Wine,Spain Rose Table Wine,Spain Rose or Blush,Spain Sparkling,Spain White Table Wine,Spiced Rum,US Chardonnay White Table Wine,US Other Cabernet Sauvignon Red Table Wine,US Other Merlot Red Table Wine,US Other Pinot Noir Red Table Wine,US Other Red Blend Red Table Wine,US Other Red Varietals Red Table Wine,US Other Shiraz or Syrah Red Table Wine,US Other White White Table Wine,Unflavored Gin,Unflavored Rum,Unflavored Vodka,Varietal Box,Vermouth Apertif Miscellaneous,Vermouth Liquor or Cordials,WA or OR Cabernet Sauvignon Red Table Wine,WA or OR Chardonnay White Table Wine,WA or OR Merlot Red Table Wine,WA or OR Other Red Varietals Red Table Wine,WA or OR Other White White Table Wine,WA or OR Pinot Grigio or Gris White Table Wine,WA or OR Pinot Noir Red Table Wine,WA or OR Red Blend Red Table Wine,WA or OR Sauvignon Blanc White Table Wine,WA or OR Shiraz or Syrah Red Table Wine,WA or OR Sparkling,WA or OR White Blend White Table Wine,White Box,White Zinfandel Rose Table Wine,White Zinfandel Rose or Blush,Wine Kegs Miscellaneous,Wine Multipack Miscellaneous
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,F

In [0]:
frequent_itemsets = fpgrowth(df_te, min_support=0.00001, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.014061,(White Box)
1,0.021523,(California Red Blend Red Table Wine)
2,0.010163,(California Merlot Red Table Wine)
3,0.003247,(New York Red Table Wine)
4,0.030307,(California Cabernet Sauvignon Red Table Wine)
...,...,...
498,0.000020,"(Unflavored Gin, Irish Whiskey)"
499,0.000015,"(Irish Whiskey, Spiced Rum)"
500,0.000029,"(Unflavored Vodka, Coffee Liquor or Cordials)"
501,0.000013,"(Coffee Liquor or Cordials, American Whiskey)"


In [0]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.005)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(WA or OR Chardonnay White Table Wine),(California Chardonnay White Table Wine),0.001559,0.028945,0.000012,0.007807,0.269729,-0.000033,0.978696
1,(Unflavored Gin),(Unflavored Vodka),0.018645,0.134195,0.000118,0.006312,0.047038,-0.002384,0.871305
2,(Nut Liquor or Cordials),(Unflavored Vodka),0.002953,0.134195,0.000018,0.006183,0.046078,-0.000378,0.871192
3,(Nut Liquor or Cordials),(American Whiskey),0.002953,0.065309,0.000017,0.005840,0.089420,-0.000176,0.940181
4,(Miscellaneous Liquor or Cordials),(American Whiskey),0.008424,0.065309,0.000053,0.006263,0.095895,-0.000497,0.940581
5,(Miscellaneous Liquor or Cordials),(Unflavored Vodka),0.008424,0.134195,0.000055,0.006504,0.048464,-0.001076,0.871473
6,(Reposado Tequila),(American Whiskey),0.003830,0.065309,0.000021,0.005563,0.085178,-0.000229,0.939920
7,(Reposado Tequila),(Unflavored Vodka),0.003830,0.134195,0.000023,0.006093,0.045402,-0.000491,0.871113
8,(Fruit Liquor or Cordials),(Unflavored Vodka),0.008083,0.134195,0.000073,0.009037,0.067345,-0.001012,0.873701
9,(Fruit Liquor or Cordials),(American Whiskey),0.008083,0.065309,0.000050,0.006150,0.094173,-0.000478,0.940475


### Use CLASS_DESC as items



In [0]:
dataset = df_by_class.CLASS_DESC
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df_te = pd.DataFrame(te_ary, columns=te.columns_)
df_te

,Box,Brandy Cognac,Cocktails Mixers,Dessert,Gift Cards,Gin,Liquor or Cordials,Miscellaneous,Ready to Drink,Red Table Wine,Rose Table Wine,Rose or Blush,Rum,Sparkling,Tequila,Vodka,Whiskey,White Table Wine
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985645,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
985646,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
985647,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
985648,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [0]:
frequent_itemsets = fpgrowth(df_te, min_support=0.00001, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.047745,(Box)
1,0.158779,(Red Table Wine)
2,0.050793,(Miscellaneous)
3,0.026440,(Rose or Blush)
4,0.030447,(Sparkling)
...,...,...
123,0.000021,"(Cocktails Mixers, Rum)"
124,0.000038,"(Cocktails Mixers, Vodka)"
125,0.000030,"(Cocktails Mixers, Whiskey)"
126,0.000020,"(Cocktails Mixers, Tequila)"


In [0]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.05)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,"(Red Table Wine, Box)",(White Table Wine),0.000210,0.124088,0.000025,0.120773,0.973287,-6.961347e-07,0.996230
1,"(Box, White Table Wine)",(Red Table Wine),0.000185,0.158779,0.000025,0.137363,0.865116,-3.954615e-06,0.975173
2,"(Red Table Wine, Miscellaneous)",(White Table Wine),0.000256,0.124088,0.000024,0.095238,0.767507,-7.375934e-06,0.968114
3,"(Miscellaneous, White Table Wine)",(Red Table Wine),0.000197,0.158779,0.000024,0.123711,0.779139,-6.902268e-06,0.959981
4,"(Red Table Wine, Rose or Blush)",(White Table Wine),0.000179,0.124088,0.000017,0.096591,0.778409,-4.909885e-06,0.969563
5,"(Rose or Blush, White Table Wine)",(Red Table Wine),0.000199,0.158779,0.000017,0.086735,0.546259,-1.432636e-05,0.921113
6,"(Red Table Wine, Sparkling)",(White Table Wine),0.000186,0.124088,0.000022,0.120219,0.968820,-7.183497e-07,0.995602
7,"(White Table Wine, Sparkling)",(Red Table Wine),0.000181,0.158779,0.000022,0.123596,0.778410,-6.353927e-06,0.959854
8,"(Liquor or Cordials, White Table Wine)",(Whiskey),0.000161,0.150494,0.000012,0.075472,0.501494,-1.210215e-05,0.918854
9,"(Whiskey, Tequila)",(Vodka),0.000215,0.178310,0.000011,0.051887,0.290992,-2.719187e-05,0.866658
